[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/neo4j-field/call-transcripts-automation/blob/main/students/3_Cleanup.ipynb)

In [15]:
%%capture
!pip --quiet install python-dotenv neo4j graphdatascience
!pip --quiet install langchain_neo4j langchain_openai langgraph

**Update Student's environment code (received via email)**

In [16]:
STUDENT_ENV_CODE="0797"

## Dotenv configuration

## Load env file from S3

In [17]:
import requests

# GCS URL
GCS_FILE_URL = "https://storage.googleapis.com/neo4j-nodes-network/atlanta/student/"+STUDENT_ENV_CODE+"/aura.txt"

# Local filename to save
LOCAL_ENV_FILE = "aura.env"

# Fetch the file and write it locally
response = requests.get(GCS_FILE_URL)
if response.status_code == 200:
    with open(LOCAL_ENV_FILE, "wb") as f:
        f.write(response.content)
    print(f"✅ Environment file downloaded and saved as {LOCAL_ENV_FILE}")
else:
    raise Exception(f"❌ Failed to fetch the .env file: {response.status_code}")

✅ Environment file downloaded and saved as aura.env


In [18]:
# NOTE: Upload files (.env)!!!!
# You can skip this cell if not using a ws.env file - alternative to above
from dotenv import load_dotenv
import os

if os.path.exists(LOCAL_ENV_FILE):
     load_dotenv(LOCAL_ENV_FILE, override=True)

     # Neo4j
     NEO4J_URI = os.getenv('NEO4J_URI')
     NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
     NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

     AURA_API_CLIENT_ID=os.getenv('AURA_API_CLIENT_ID')
     AURA_API_CLIENT_SECRET=os.getenv('AURA_API_CLIENT_SECRET')

     IS_AURA = os.environ.get("AURA", True) is True

     # AI
     LLM = 'gpt-4o'
     AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')

print(f"OPENAI_KEY: {AZURE_OPENAI_API_KEY}")

OPENAI_KEY: 6CgahPtoFmvzqh2gsBzef795O95YTMrIHg4KqNjYfOjvbC3BTdXmJQQJ99BCACYeBjFXJ3w3AAABACOGtZuK


## Connecting to Neo4j

Provide your Neo4j credentials. We need the DB conection URL, the username (probably neo4j), and your password.

In [19]:
# # username is neo4j by default
# NEO4J_USERNAME = 'neo4j'

# # You will need to change these to match your credentials
# NEO4J_URI = 'neo4j+s://a51c4bb3.databases.neo4j.io'
# NEO4J_PASSWORD = 'R50r64jHpBzFgxZS6tttK7MXwO5agK6syL-Pznw5m_Y'

In [20]:
from langchain_neo4j import Neo4jGraph

# You can skip this cell if not using a ws.env file - alternative to above
# Get credentials from .env file
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

graph_db = Neo4jGraph(url=os.getenv("NEO4J_URI"), username=os.getenv("NEO4J_USERNAME"), password=os.getenv("NEO4J_PASSWORD"))

In [21]:
# from graphdatascience import GraphDataScience

# gds = GraphDataScience(
#     NEO4J_URI,
#     auth=(NEO4J_USERNAME, NEO4J_PASSWORD),
#     aura_ds=False
# )
# gds.set_database('neo4j')

# Cleanup Post Ingestion

In [22]:
graph_db.query("""
  MATCH (n:Observation|ProcessElement)
  CALL (n) {
    DETACH DELETE n
  } IN TRANSACTIONS OF 1000 ROWS
  return count(n) as deleted
  """)

[{'deleted': 1089}]

# Cleanup everything

In [23]:
graph_db.query("""
  MATCH (n)
  CALL (n) {
    DETACH DELETE n
  } IN TRANSACTIONS OF 1000 ROWS
  return count(n) as deleted
  """)

[{'deleted': 922}]

# Drop Constraints

In [24]:
graph_db.query("DROP CONSTRAINT commentId IF EXISTS")
graph_db.query("DROP CONSTRAINT callId IF EXISTS")
graph_db.query("DROP CONSTRAINT entityName IF EXISTS")
graph_db.query("DROP CONSTRAINT customerId IF EXISTS")
graph_db.query("DROP CONSTRAINT representativeId IF EXISTS")
graph_db.query("DROP CONSTRAINT observationId IF EXISTS")
graph_db.query("DROP CONSTRAINT processElementId IF EXISTS")

[]